In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,2"

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
tnts_san = SAN(TS=True, TN=False, depth=2)
tnts_san.compile(optimizer='adam', loss='mae')
tnts_san(np.random.random((1, 160, 36, 36, 3)));
tnts_san.summary()

Model: "san"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Sample 0 (Sequential)           │ (160, 18, 18, 64)      │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sample 1 (Sequential)           │ (160, 9, 9, 32)        │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 0 (Sequential)            │ (160, 36, 36, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 1 (Sequential)            │ (160, 18, 18, 16)      │        11,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 0 (Sequential)            │ (160, 18, 18, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 1 (Sequential)            │ (160, 9, 9, 32)        │        13,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 0 (Sequential)        │ (160, 36, 36, 16)      │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 1 (Sequential)        │ (160, 18, 18, 8)       │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 0 (Sequential)        │ (160, 18, 18, 32)      │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 1 (Sequential)        │ (160, 9, 9, 16)        │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 0 (Sequential)            │ (160, 36, 36, 1)       │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 1 (Sequential)            │ (160, 18, 18, 1)       │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 0 (Sequential)            │ (160, 18, 18, 1)       │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 1 (Sequential)            │ (160, 9, 9, 1)         │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pooling & Downsample            │ (160, 18, 18, 32)      │             0 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten & Dense (Sequential)    │ (160, 1)               │        65,793 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,277 (817.49 KB)

 Trainable params: 209,277 (817.49 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x36x36_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)
#training_set = training_set.apply_fn(gnoise_aug)

rlap_160x36x36_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x36x36_all             datatape has been loaded.     2944 items total. fold=val&extended_hr=False selected


In [4]:
#tnts_san.fit(training_set, validation_data=validation_set, epochs=20, callbacks=[keras.callbacks.ModelCheckpoint('../weights/ts_san_36.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(tnts_san, training_set, validation_set, epochs=20, check_point_path='../weights/ts_san_36.weights.h5')
tnts_san.load_weights('../weights/ts_san_36.weights.h5')

Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.8643	Validation loss:0.8646	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.8606	Validation loss:0.8645	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.8605	Validation loss:0.8645	


In [5]:
eval_on_dataset(dataset_H5_mmpd, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TS-SAN-F2W_RLAP_MMPD.h5')
get_metrics('../results/TS-SAN-F2W_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,33.477±0.238,39.11±4.316,37.986±0.223,0.00902
Whole,31.707±0.657,35.9±6.992,35.767±0.567,0.03481


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TS-SAN-F2W_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/TS-SAN-F2W_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,23.362±0.361,27.996±4.427,31.775±0.421,-0.09944
Whole,21.417±1.054,25.315±7.435,28.554±1.115,-0.25154


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TS-SAN-F2W_RLAP_PURE.h5')
get_metrics('../results/TS-SAN-F2W_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,23.408±0.886,33.853±8.719,30.567±0.983,0.01832
Whole,17.834±2.847,28.219±15.196,21.264±2.338,0.04904


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TS-SAN-F2W_RLAP_UBFC.h5')
get_metrics('../results/TS-SAN-F2W_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,38.783±1.27,48.421±10.073,37.562±1.131,0.01036
Whole,38.045±3.045,42.859±15.719,37.106±2.451,0.06259
